<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [30]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [31]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [32]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [33]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [34]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [35]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [36]:
df.Burrito.value_counts()

California     169
Other          156
Asada           43
Surf & Turf     28
Carnitas        25
Name: Burrito, dtype: int64

In [37]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df = df.set_index('Date')
# df.dropna('NonSD', 'Lettuce',	'Tomato',	'Bell peper',	'Carrots', inplace=True)
df = df.fillna(0)
df = df.replace('x', 1)
df = df.replace('X', 1)
print(df.shape)
df.head()

(421, 58)


,Burrito,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-18,California,3.5,4.2,0,6.49,3.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False
2016-01-24,California,3.5,3.3,0,5.45,3.5,0.0,0.0,0.0,0.0,0.0,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False
2016-01-24,Carnitas,0.0,0.0,0,4.85,1.5,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False
2016-01-24,Asada,0.0,0.0,0,5.25,2.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False
2016-01-27,California,4.0,3.8,1,6.59,4.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,True


# 1. Set **target vector** and separate it from the **feature matrix**.

In [38]:
target = 'Great'
y = df[target]
X = df.drop([target]+['Chips', 'Yelp', 'Google', 'Mass (g)', 'Density (g/mL)', 
                      'Unreliable', 'Chicken',	'Shrimp',	'Fish',	'Rice',	
                      'Beans',	'Lettuce',	'Tomato',	'Bell peper',	'Carrots',	
                      'Cabbage',	'Sauce',	'Salsa.1',	'Cilantro',	'Onion',	
                      'Taquito',	'Pineapple',	'Ham',	'Chile relleno',	
                      'Nopales',	'Lobster',	'Queso',	'Egg',	'Mushroom',	
                      'Bacon',	'Sushi', 'Avocado',	'Corn',	'Zucchini','NonSD', 
                      'Pork', 'Length', 'Circum', 'Volume'], axis=1)


print(y.shape)
print(X.shape)

(421,)
(421, 18)


# 2. Split our **training** and **validation** - and **testing** sets

In [39]:
#  Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
# Should I create sepearate dataframes for training, validation, and test sets?
cutoff1 = '2017-01-01'
cutoff2 = '2018-01-01'
mask1 = X.index < cutoff1
mask2 = (X.index >= cutoff1) & (X.index < cutoff2)
mask3 = (X.index >= cutoff2)

X_train, y_train = X.loc[mask1], y.loc[mask1]
X_val, y_val = X.loc[mask2], y.loc[mask2]

In [40]:
# assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

#3. Establish Baseline

In [41]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline Accuracy: 0.5906040268456376


#Create the Model
What sorts of transformations do we need for out model to work?

-Encode categorical variables: `OneHotEncoder`

-Fill in missing values(`NaN`): `SimpleImputer`

In [42]:
# Make a pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

Step1: Instantiate the pipeline

In [43]:
model = make_pipeline(
    OneHotEncoder(), # outputs a dataframe
    SimpleImputer(), # outputs a numpy array
    LogisticRegression() # Only one predictor and it has to be the end
)

Step2: Fit Model to the training data

In [60]:
model.fit(X_train, y_train);

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#Check Metrics

In [45]:
# Check metrics for accuracy 
print('Training Accuracy:', model.score(X_train, y_train))
print('Validation Accuracy:', model.score(X_val, y_val))

Training Accuracy: 0.8926174496644296
Validation Accuracy: 0.7529411764705882


In [46]:
from sklearn.metrics import accuracy_score

Step3: Predict

In [59]:
y_pred = model.predict(X_val)
print('Accuracy Score:', accuracy_score(y_val, y_pred))

Accuracy Score: 0.7529411764705882


In [51]:
y_pred_proba = model.predict_proba(X_val)

In [53]:
y_pred_proba.shape

(85, 2)

In [67]:
y_pred_proba[:5]

array([[0.31929305, 0.68070695],
       [0.97979337, 0.02020663],
       [0.27046265, 0.72953735],
       [0.37659317, 0.62340683],
       [0.99861234, 0.00138766]])

#Run model on test data

In [61]:
# This is the test data from earlier
mask3 = (X.index >= cutoff2)
X_test, y_test = X.loc[mask3], y.loc[mask3]

In [62]:
print(X_test.shape)
print(y_test.shape)

(38, 18)
(38,)


In [64]:
print('Training Accuracy:', model.score(X_train, y_train))
print('Validation Accuracy:', model.score(X_val, y_val))
print('Test Accuracy:', model.score(X_test, y_test))

Training Accuracy: 0.8926174496644296
Validation Accuracy: 0.7529411764705882
Test Accuracy: 0.7631578947368421
